In [1]:
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../src')

from dataset import DatasetConfig, DatasetBuilder
import os
from functools import partial
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from collections import defaultdict
from typing import Callable, Dict, List, Tuple
%matplotlib inline

In [25]:
ds_config =  DatasetConfig(model_dir='../dataset/pinecone/sparse/0/',
                           images_dir='../dataset/pinecone/images/',
                           batch_from_single_image=True,
                           is_training=True,
                           batch_size=8)

In [26]:
dataset = DatasetBuilder(ds_config)
ds = dataset.build()

In [27]:
lines = []
colors = []

def make_line(origin, direction, length=10):
    return origin, origin + direction * length

for elements in ds.take(200).as_numpy_iterator():
    origins = elements["origins"]
    directions = elements["directions"]
    pixels = elements["pixels"]
    N = origins.shape[0]
    for i in range(N):
        p0, p1 = make_line(origins[i, ...], directions[i, ...])
        lines.append(p0)
        lines.append(p1)
        colors.append(pixels[i, ...])
    print("Done")

Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done


In [13]:
data = np.asarray(lines)
np.save('lines.npy', data)

colors = np.asarray(colors)
np.save('colors.npy', colors)

In [28]:
[lines[i] for i in range(0, len(lines), 2)]

[array([ 2.6899767, -2.4331675,  1.3265511], dtype=float32),
 array([ 2.6899767, -2.4331675,  1.3265511], dtype=float32),
 array([ 2.6899767, -2.4331675,  1.3265511], dtype=float32),
 array([ 2.6899767, -2.4331675,  1.3265511], dtype=float32),
 array([ 2.6899767, -2.4331675,  1.3265511], dtype=float32),
 array([ 2.6899767, -2.4331675,  1.3265511], dtype=float32),
 array([ 2.6899767, -2.4331675,  1.3265511], dtype=float32),
 array([ 2.6899767, -2.4331675,  1.3265511], dtype=float32),
 array([-2.9017558 ,  1.3308653 ,  0.09751619], dtype=float32),
 array([-2.9017558 ,  1.3308653 ,  0.09751619], dtype=float32),
 array([-2.9017558 ,  1.3308653 ,  0.09751619], dtype=float32),
 array([-2.9017558 ,  1.3308653 ,  0.09751619], dtype=float32),
 array([-2.9017558 ,  1.3308653 ,  0.09751619], dtype=float32),
 array([-2.9017558 ,  1.3308653 ,  0.09751619], dtype=float32),
 array([-2.9017558 ,  1.3308653 ,  0.09751619], dtype=float32),
 array([-2.9017558 ,  1.3308653 ,  0.09751619], dtype=float32),
